<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Canales%20de%20Youtube/Python%20Engineer/AdaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AdaBoost**

Este algoritmo se basa en ir agrupando otros algoritmos de clasificacion, para que en conjunto generen una prediccion.

Asimismo y, a diferencia del algoritmo de Random Forest, es que el **voto** de cada estimador no valen lo mismo, es decir, existe un grado de importancia (**weight**) entre los estimadores que siendo estos ponderados por sus votos es que generan la prediccion del algoritmo.

**Weak Learner (Decision Stump)**

Es un algoritmo que sencillamente clasifica los datos segun un limite (similar a uno de los pasos del algoritmo de Decision Tree)

**Error**

- Primera itereacion

$$
ϵ_{1} = \frac{desaceirtos}{N}
$$

- A partir de la segunda iteracion

$$
ϵ_{t} = \sum weights
$$

Nota: Si el error es mayor a 0.5, se itercambia la clasificacion y se calcula el $error = 1 - error$

**Weights**

- Al inicio
$$
w_{0} = \frac{1}{N} para cada muestra
$$

- Luego

$$
w = \frac{w \cdot e^{- αyh(X)}}{\sum w} 
$$

**Performance**

$$
\alpha = 0.5 \cdot log(\frac{1-ϵ_{t}}{ϵ_{t}})
$$

**Prediction**

$$
y = sign(\sum_{t}^{T} α_{t} \cdot h(X))
$$

**Training**

Se inicializan los pesos de cada mustra en $\frac{1}{N}$

- Entrenamos a un clasificador debil (se busca la mejor variable y limite para segmentar)
- Calculamos el error $ϵ_{t} = \sum_{desaciertos} weights$
 - Cambiar el error y la polaridad si este es mayor a 0.5
- Calcular $ \alpha = 0.5 \cdot log(\frac{1 - \epsilon_{t}}{ϵ_{t}})$
- Actualizar los pesos: $w = \frac{w \cdot e^{- αh(X)}}{Z}$


## **Armando el algoritmo desde cero**

In [1]:
# Importamos la biblioteca
import numpy as np

In [2]:
# clasificador debil (DecisionStump)
class DecisionStump:
  def __init__(self):
    self.polarity = 1
    self.feature_idx = None
    self.threshold = None 
    self.alpha = None 

  def predict(self, X):
    n_samples = X.shape[0]
    X_column = X[:, self.feature_idx]
    predictions = np.ones(n_samples)
    if self.polarity == 1:
      predictions[X_column < self.threshold] = -1
    else:
      predictions[X_column > self.threshold] = -1

    return predictions

class Adaboost:
  def __init__(self, n_clf=5):
    self.n_clf = n_clf
    self.clfs = []

  def fit(self, X, y):
    n_samples, n_features = X.shape

    # Inicializamos los pesos (1/N)
    w = np.full(n_samples, (1/n_samples))

    self.clfs = []

    # Iteramos de clasificador en clasificador
    for _ in range(self.n_clf):
      clf = DecisionStump()
      min_error = float("inf")

      # Busqueda del limite y la variable
      for feature_i in range(n_features):
        X_column = X[:, feature_i]
        thresholds = np.unique(X_column) 

        for threshold in thresholds:
          # Predecimos con polaridad 1
          p = 1
          predictions = np.ones(n_samples)
          predictions[X_column < threshold] = -1

          # Error = suma de los pesos de los desaciertos
          misclassified = w[y != predictions]
          error = sum(misclassified)

          if error > 0.5:
            error = 1 - error 
            p = -1

          # Guardar la mejor configuracion
          if error < min_error:
            clf.polarity = p 
            clf.threshold = threshold
            clf.feature_idx = feature_i
            min_error = error

    # Calcular el alpha
    EPS = 1e-10 
    clf.alpha = 0.5*np.log((1.0 - min_error + EPS)/(min_error + EPS))

    # Calcular las predicciones y actualizar los pesos 
    predictions = clf.predict(X)

    w *= np.exp(-clf.alpha * y * predictions)

    # Noramlizamos a 1
    w /= np.sum(w)

    # Guardamos el clasificador
    self.clfs.append(clf)

  def predict(self, X):
    clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
    y_pred = np.sum(clf_preds, axis=0)
    y_pred = np.sign(y_pred)

    return y_pred

## **Ahora probemos el algorimto**

In [3]:
# Importamos las bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [4]:
# definimos una funcion para medir el rendimiento
def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true == y_pred) / len(y_true)
  return accuracy

# Cargamos los datos
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# Adaptamos el un poco el problema para el algoritmo
y[y == 0] = -1

# Separamos en set de entrenamiento y validacion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# Adaboost classification con 5 clasificadores debiles
clf = Adaboost(n_clf=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy(y_test, y_pred)
acc

0.9122807017543859

## **Ahora probemos desde sklearn**


In [5]:
# Importamos el clasificador
from sklearn.ensemble import AdaBoostClassifier as ABC

In [8]:
# Cargamos el clasificador, lo entrenamos y lo evaluamos
clf_sk = ABC(n_estimators=5, random_state=5)
clf_sk.fit(X_train, y_train)
y_pred_sk = clf_sk.predict(X_test)

acc_sk = accuracy(y_test, y_pred_sk)
acc_sk

0.9649122807017544